<a href="https://colab.research.google.com/github/ahassanzadeh/Transaction_Value_Prediction/blob/master/True_amount_cents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting customer Transactions amount 

How much transaction company should excpect for the next month, quarter and year? 

This questions is categorized under Time series forcasting which is one machine learning branches. they are two major models for time series forcasting are:
# 1- Autoregressive intergrated moving average(ARIMA) [Classic approach]

This model is stand for Auto Regressive moving average(ARMA) model. 
Major parameters are:
- P => Periods to lag for eg: (if P= 3 then we will use the three previous periods of our time series in the autoregressive portion of the calculation).P helps adjust the line that is being fitted to forecast the series
 - **Purely autoregressive models resemble a linear regression where the predictive variables are P number of previous periods**
- D => In an ARIMA model we transform a time series into stationary one(series without trend or seasonality) using differencing. **Differencing is a method of transforming a non-stationary time series into a stationary one. This is an important step in preparing data to be used in an ARIMA model.** D refers to the number of differencing transformations required by the time series to get stationary.
- Q => This variable denotes the lag of the error component, where error component is a part of the time series not explained by trend or seasonality. **The Autocorrelation function plot will let you know how the given time series is correlated with itself**

- m => refers to the number of periods in each season(for seasonal time-series)





## Steps: 

- Step 1 — Check stationarity: If a time series has a trend or seasonality component, it must be made stationary before we can use ARIMA to forecast.
- Step 2 — Difference: If the time series is not stationary, it needs to be stationarized through differencing. Take the first difference, then check for stationarity. Take as many differences as it takes. Make sure you check seasonal differencing as well.
**Stationary time series is when the mean and variance are constant over time. It is easier to predict when the series is stationary.**

- Step 3 — Filter out a validation sample: This will be used to validate how accurate our model is. Use train test validation split to achieve this. 
- Step 4 — Select AR and MA terms: Use the ACF and PACF to decide whether to include an AR term(s), MA term(s), or both.Normally in an ARIMA model, we make use of either the AR term or the MA term:
 - If there is a Positive autocorrelation at lag 1 then we use the AR model
 - If there is a Negative autocorrelation at lag 1 then we use the MA model
- Step 5 — Build the model: Build the model and set the number of periods to forecast to N (depends on your needs).
- Step 6 — Validate model: Compare the predicted values to the actuals in the validation sample.



# 2- Long short-term memory(LSTM)[Modern Approach]
LSTM is one the most recent deep learning approaches make of autoregressive Recurrent Nueral Network(RNN)  

LSTM is used in this notebook as follows: 

In [1]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# import Plotly for high quality visulization:
!pip install chart_studio  
!pip install plotly==4.8.2
import chart_studio.plotly as py
import plotly.graph_objects as go

In [4]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

Using TensorFlow backend.


## Data management 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path = "/content/drive/My Drive/Projects/Digit/train_val.csv"
df_sales= pd.read_csv(path)
# df_sales= pd.read_csv(uploaded)
df_sales.columns=['CustomerID','date', 'sales']

In [7]:
df_sales.shape

(1048575, 3)

In [8]:
df_sales.head(10)

,CustomerID,date,sales
0,54b80cb0cb084b0c00294f2a|6ce908fc9895fdfc035e4...,2018-06-08T00:00:00.000Z,538.0
1,54bde3073deefe0c00a01401|46e37148a68739387c48b...,2017-11-28T00:00:00.000Z,1614.0
2,54c6ea8213078c0c001cec71|044a408e374c8a102b709...,2018-04-02T00:00:00.000Z,22105.0
3,54e643cd95962f0c004cde71|ee74f9800708efd6b39c3...,2018-05-29T00:00:00.000Z,7803.0
4,54e64674de3f910c0014d567|bac357ccd7c020cf540e8...,2017-12-07T00:00:00.000Z,25.0
5,54e659bc2fb32c0c007c030f|8816cf3aede474ecbdb03...,2017-10-18T00:00:00.000Z,2381.0
6,54e67cc651c9f70c00d49a8b|6a9ac4aefba74c038a149...,2018-03-01T00:00:00.000Z,9990.0
7,54e693f37216210c0042db88|33c26323987617afdcc82...,2018-04-16T00:00:00.000Z,2500.0
8,54e699308eac6c0c00160cfe|b83c984368bdf9354e147...,2018-06-04T00:00:00.000Z,6098.0
9,54e69a1d910e5e0c0058bb98|0c978d2645c67c16857d0...,2017-11-21T00:00:00.000Z,632.0


In [9]:
df_sales['date'] = df_sales['date']. astype(str).str[:-14]
df_sales['date'] = pd.to_datetime(df_sales['date'], errors='coerce')
# df_sales.sort_values(df_sales['date'], axis=1)

df_sales;

In [10]:
#represent month in date field as its first day
print(df_sales['date'].dt.year.astype('str').str[:4] + '-' + df_sales['date'].dt.month.astype('str') + '-01')
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'


0           2018-6.0-01
1          2017-11.0-01
2           2018-4.0-01
3           2018-5.0-01
4          2017-12.0-01
               ...     
1048570      nan-nan-01
1048571      nan-nan-01
1048572      nan-nan-01
1048573      nan-nan-01
1048574      nan-nan-01
Name: date, Length: 1048575, dtype: object


In [11]:
#groupby date and sum the sales
df_sales = df_sales.groupby('date').sales.sum().reset_index()

In [12]:
df_sales.head()

,date,sales
0,2017.0-10.0-01,481318803.0
1,2017.0-11.0-01,567076518.0
2,2017.0-12.0-01,683587990.0
3,2017.0-9.0-01,107578272.0
4,2018.0-1.0-01,724746118.0


## Stationary/Non-stationary data 

In this section, three steps are followered to conver the Non-sstationary data 
to stationary. 

In [13]:
#plot monthly sales
plot_data = [
    go.Scattergl(
        x=df_sales['date'],
        y=df_sales['sales'],
    )
]

plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()


Obviously, it is not stationary and has an increasing trend over the months. One method is to get the difference in sales compared to the previous month and build the model on it:



In [14]:
#create a new dataframe to model the difference
df_diff = df_sales.copy()

#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(1)

#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

Now we have the required dataframe for modeling the difference:


In [15]:
df_diff.head(10)

,date,sales,prev_sales,diff
1,2017.0-11.0-01,567076518.0,481318803.0,85757715.0
2,2017.0-12.0-01,683587990.0,567076518.0,116511472.0
3,2017.0-9.0-01,107578272.0,683587990.0,-576009718.0
4,2018.0-1.0-01,724746118.0,107578272.0,617167846.0
5,2018.0-2.0-01,714481839.0,724746118.0,-10264279.0
6,2018.0-3.0-01,804230366.0,714481839.0,89748527.0
7,2018.0-4.0-01,743952976.0,804230366.0,-60277390.0
8,2018.0-5.0-01,593491534.0,743952976.0,-150461442.0
9,2018.0-6.0-01,138480480.0,593491534.0,-455011054.0
10,nan-nan-01,20066161.0,138480480.0,-118414319.0


Let’s plot it and check if it is stationary now:

In [16]:
#plot sales diff
plot_data = [
    go.Scattergl(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]

plot_layout = go.Layout(
        title='Montly Sales Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()

## Looking at previous monthly data 


In [17]:
#create new dataframe from time series 
df_supervised = df_diff.drop(['prev_sales'],axis=1)

#adding lags
for inc in range(1,3):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

Check out our new dataframe called df_supervised:



In [18]:
df_supervised.head(100)

,date,sales,diff,lag_1,lag_2
1,2017.0-11.0-01,567076518.0,85757715.0,NaN,NaN
2,2017.0-12.0-01,683587990.0,116511472.0,85757715.0,NaN
3,2017.0-9.0-01,107578272.0,-576009718.0,116511472.0,85757715.0
4,2018.0-1.0-01,724746118.0,617167846.0,-576009718.0,116511472.0
5,2018.0-2.0-01,714481839.0,-10264279.0,617167846.0,-576009718.0
6,2018.0-3.0-01,804230366.0,89748527.0,-10264279.0,617167846.0
7,2018.0-4.0-01,743952976.0,-60277390.0,89748527.0,-10264279.0
8,2018.0-5.0-01,593491534.0,-150461442.0,-60277390.0,89748527.0
9,2018.0-6.0-01,138480480.0,-455011054.0,-150461442.0,-60277390.0
10,nan-nan-01,20066161.0,-118414319.0,-455011054.0,-150461442.0


In [19]:
df_supervised.tail(6)

,date,sales,diff,lag_1,lag_2
5,2018.0-2.0-01,714481839.0,-10264279.0,617167846.0,-576009718.0
6,2018.0-3.0-01,804230366.0,89748527.0,-10264279.0,617167846.0
7,2018.0-4.0-01,743952976.0,-60277390.0,89748527.0,-10264279.0
8,2018.0-5.0-01,593491534.0,-150461442.0,-60277390.0,89748527.0
9,2018.0-6.0-01,138480480.0,-455011054.0,-150461442.0,-60277390.0
10,nan-nan-01,20066161.0,-118414319.0,-455011054.0,-150461442.0


In [20]:
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)
df_supervised.head()

,date,sales,diff,lag_1,lag_2
0,2017.0-9.0-01,107578272.0,-576009718.0,116511472.0,85757715.0
1,2018.0-1.0-01,724746118.0,617167846.0,-576009718.0,116511472.0
2,2018.0-2.0-01,714481839.0,-10264279.0,617167846.0,-576009718.0
3,2018.0-3.0-01,804230366.0,89748527.0,-10264279.0,617167846.0
4,2018.0-4.0-01,743952976.0,-60277390.0,89748527.0,-10264279.0


In [21]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)


# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

-0.010878840961481506


In [22]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 ', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

-0.21303717997758298


In [23]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date'],axis=1)

R-squared results is low therefore it shows this regression is not the best method to fit the data as the error is high. however it still applicable to our approach.  

In [24]:
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [25]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   diff    8 non-null      float64
 1   lag_1   8 non-null      float64
 2   lag_2   8 non-null      float64
dtypes: float64(3)
memory usage: 320.0 bytes


In [26]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

## LSTM Model

In [27]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [28]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [29]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=500, batch_size=1, verbose=1, shuffle=False)

Epoch 1/500
2/2 [==============================] - 1s 253ms/step - loss: 0.9201
Epoch 2/500
2/2 [==============================] - 0s 6ms/step - loss: 0.9096
Epoch 3/500
2/2 [==============================] - 0s 7ms/step - loss: 0.9005
Epoch 4/500
2/2 [==============================] - 0s 7ms/step - loss: 0.8932
Epoch 5/500
2/2 [==============================] - 0s 7ms/step - loss: 0.8872
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 0.8820
Epoch 7/500
2/2 [==============================] - 0s 6ms/step - loss: 0.8773
Epoch 8/500
2/2 [==============================] - 0s 10ms/step - loss: 0.8728
Epoch 9/500
2/2 [==============================] - 0s 8ms/step - loss: 0.8685
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 0.8643
Epoch 11/500
2/2 [==============================] - 0s 8ms/step - loss: 0.8602
Epoch 12/500
2/2 [==============================] - 0s 10ms/step - loss: 0.8562
Epoch 13/500
2/2 [==============================] - 0s 8m

In [30]:
y_pred = model.predict(X_test,batch_size=1)

In [31]:
y_pred

array([[ 0.11693785],
       [ 2.6367238 ],
       [ 0.12906875],
       [-0.6472174 ],
       [-0.09083344],
       [ 0.02351336]], dtype=float32)

In [32]:
y_test

array([[-0.0516995 ],
       [ 0.11594161],
       [-0.1355313 ],
       [-0.28669749],
       [-0.79718247],
       [-0.23298022]])

In [33]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

In [34]:
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print (np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

[[  0.11693785   2.44589474 -44.03652611]]
[[ 2.63672376  0.63387185 33.55903817]]
[[ 0.12906875  0.92270866 -7.24456999]]
[[-0.64721739  0.48943407 -0.7404667 ]]
[[ -0.09083344   0.22898269 -10.49705787]]
[[  0.02351336  -0.65055607 -16.3619707 ]]


In [35]:
pred_test_set[0]

array([[  0.11693785,   2.44589474, -44.03652611]])

In [36]:
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

In [37]:
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [38]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].date)
act_sales = list(df_sales[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [39]:
df_result

,pred_value,date
0,815088989,2018.0-2.0-01
1,2308100717,2018.0-3.0-01
2,901810396,2018.0-4.0-01
3,378409403,2018.0-5.0-01
4,559880386,2018.0-6.0-01
5,173087349,nan-nan-01


In [40]:
df_sales.head()

,date,sales
0,2017.0-10.0-01,481318803.0
1,2017.0-11.0-01,567076518.0
2,2017.0-12.0-01,683587990.0
3,2017.0-9.0-01,107578272.0
4,2018.0-1.0-01,724746118.0


In [41]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')

In [42]:
df_sales_pred

,date,sales,pred_value
0,2017.0-10.0-01,481318803.0,NaN
1,2017.0-11.0-01,567076518.0,NaN
2,2017.0-12.0-01,683587990.0,NaN
3,2017.0-9.0-01,107578272.0,NaN
4,2018.0-1.0-01,724746118.0,NaN
5,2018.0-2.0-01,714481839.0,8.150890e+08
6,2018.0-3.0-01,804230366.0,2.308101e+09
7,2018.0-4.0-01,743952976.0,9.018104e+08
8,2018.0-5.0-01,593491534.0,3.784094e+08
9,2018.0-6.0-01,138480480.0,5.598804e+08


In [43]:
# plot actual and predicted
plot_data = [
    go.Scattergl(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]

plot_layout = go.Layout(
        title='Transaction amount(cents)'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.show()


The fitting is not ideal and in the future, combination of convolotional nueral network and LSTM and adding new features to the model can be useful. 
At end, by using this model, we have a baseline for predictiong the transactions in the future. 